## Switch to CPU Instance (Advisable only for Non Colab-Pro instance)

1. Switch to CPU Instance for until Step 3 for non GPU dependent tasks
2. This increases your time available for the GPU dependent tasks on a Colab instance
2. Change Runtime type to CPU by Runtime(Top Left tab)->Change Runtime Type->None(Hardware Accelerator)
3.   Then click on Connect (Top Right)



## Mounting Google drive
Mount your Google drive storage to this Colab instance

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Binary Semantic Segmentation using TAO UNET

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. 

Train Adapt Optimize (TAO) Toolkit  is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

<img align="center" src="https://developer.nvidia.com/sites/default/files/akamai/embedded-transfer-learning-toolkit-software-stack-1200x670px.png" width="1080"> 

## Learning Objectives
In this notebook, you will learn how to leverage the simplicity and convenience of TAO to:

* Take a pretrained resnet18 model and train a ResNet-18 UNet model on the ISBI dataset
* Run Inference on the trained model and visualize the inferences
* Export the trained model to a .etlt file for deployment to DeepStream
* Run inference on the exported .etlt model to verify deployment using TensorRT

### Table of Contents

This notebook shows an example use case of UNet Binary Semantic Segmentation using Train Adapt Optimize (TAO) Toolkit.

0. [Set up env variables](#head-0)
1. [Prepare dataset and pre-trained model](#head-1) <br>
    1. [Verify downloaded dataset](#head-1-1)
    2. [Prepare ISBI data from TIF](#head-1-2)
    3. [Visualize the Groundtruth Masks](#head-1-3)
    4. [Download pre-trained model](#head-1-4)
2. [Setup GPU environment](#head-2) <br>
    2.1 [Connect to GPU Instance](#head-2-1) <br>
    2.2 [Mounting Google drive](#head-2-2) <br>
    2.3 [Setup Python environment](#head-2-3) <br>
    2.4 [Reset env variables](#head-2-4) <br>
3. [Provide training specification](#head-3)
4. [Run TAO training](#head-4)
5. [Evaluate trained models](#head-5)
6. [Visualizing Inferences](#head-6)
7. [Prune trained models](#head-7)
8. [Retrain pruned models](#head-8)
9. [Evaluate retrained model](#head-9)

## 0. Set up env variables <a class="anchor" id="head-0"></a>

When using the purpose-built pretrained models from NGC, please make sure to set the `$KEY` environment variable to the key as mentioned in the model overview. Failing to do so, can lead to errors when trying to load them as pretrained models.

*Note: Please make sure to remove any stray artifacts/files from the `$USER_EXPERIMENT_DIR` or `$DATA_DOWNLOAD_DIR` paths as mentioned below, that may have been generated from previous experiments. Having checkpoint files etc may interfere with creating a training graph for a new experiment.*

*Note: This notebook currently is by default set up to run training using 1 GPU. To use more GPU's please update the env variable `$NUM_GPUS` accordingly*


In [ ]:
# Setting up env variables for cleaner command line commands.
import os

%set_env KEY=nvidia_tlt
%set_env GPU_INDEX=0

# Change the paths according to your directory structure
%set_env EXPERIMENT_DIR=/results/unet
%set_env DATA_DIR=/content/drive/MyDrive/unet_data
%set_env SPECS_DIR=/content/drive/MyDrive/ColabNotebooks/tensorflow/unet/specs

! ls -l $SPECS_DIR

## 1. Prepare dataset and pre-trained model <a class="anchor" id="head-1"></a>

We will be using the `ISBI Challenge: Segmentation of neuronal structures in EM stacks` dataset for the binary segmentation tutorial. The data is present in this github repository https://github.com/alexklibisz/isbi-2012/tree/master/data. The data is in .tif format. Copy the train-labels.tif, train-volume.tif, test-volume.tif files to `$DATA_DIR/isbi`. 

### A. Verify downloaded dataset <a class="anchor" id="head-1-1"></a>

In [ ]:
# Check the .tiff files are present
!mkdir -p $DATA_DIR/isbi
!ls -l $DATA_DIR
!if [ ! -f $DATA_DIR/isbi/test-volume.tif ]; then echo 'test-volume.tif file not found, please download.'; else echo 'Found test-volume.tif file.';fi
!if [ ! -f $DATA_DIR/isbi/train-labels.tif ]; then echo 'train-labels file not found, please download.'; else echo 'Found train-labels.tif file.';fi
!if [ ! -f $DATA_DIR/isbi/train-volume.tif ]; then echo 'train-volume.tif file not found, please download.'; else echo 'Found train-volume.tif file.';fi

### B.  Prepare ISBI data from TIF <a class="anchor" id="head-1-2"></a>

Prepare the images and masks from the .tif files by running the following script.

In [ ]:
%cd /content/drive/MyDrive/ColabNotebooks/tensorflow/unet/
!bash prepare_data.sh $DATA_DIR/isbi

In [ ]:
!if [ ! -f $DATA_DIR/images/image_0.png ]; then echo 'Data prepared successfully !'; else echo 'Please verify Data Preparation.';fi
# Checking the files
!ls -l $DATA_DIR/isbi
!ls -l $DATA_DIR/isbi/images
!ls -l $DATA_DIR/isbi/masks
!ls -l $DATA_DIR/isbi/images/train

### C.  Visualize the Groundtruth Masks <a class="anchor" id="head-1-3"></a>

In [ ]:
# Run this cell to visualize the training data and the masks
%cd /content/drive/MyDrive/ColabNotebooks/tensorflow/unet/
!python3 vis_annotation.py -i $DATA_DIR/isbi/images/train -m $DATA_DIR/isbi/masks/train -o $EXPERIMENT_DIR/isbi_experiment_unpruned/vis_gt --num_classes 2 --num_images 10

In [ ]:
# Simple grid visualizer
%matplotlib inline

import matplotlib.pyplot as plt
import os
from math import ceil
valid_image_ext = ['.jpg', '.png']

def visualize_images(image_dir, result_dir, num_cols=4, num_images=10):
    output_path = os.path.join(os.environ['EXPERIMENT_DIR'], result_dir, image_dir)
    num_rows = int(ceil(float(num_images) / float(num_cols)))
    f, axarr = plt.subplots(num_rows, num_cols, figsize=[80,30])
    f.tight_layout()
    a = [os.path.join(output_path, image) for image in os.listdir(output_path) 
         if os.path.splitext(image)[1].lower() in valid_image_ext]
    for idx, img_path in enumerate(a[:num_images]):
        col_id = idx % num_cols
        row_id = idx // num_cols
        img = plt.imread(img_path)
        axarr[row_id, col_id].imshow(img) 

In [ ]:
# Visualizing the sample images.
OUTPUT_PATH = 'vis_gt' # relative path from $USER_EXPERIMENT_DIR.
COLS = 2 # number of columns in the visualizer grid.
IMAGES = 4 # number of images to visualize.

visualize_images(OUTPUT_PATH, "isbi_experiment_unpruned", num_cols=COLS, num_images=IMAGES)

### D. Download pre-trained model <a class="anchor" id="head-1-4"></a>

We will use NGC CLI to get the pre-trained models. For more details, go to ngc.nvidia.com and click the SETUP on the navigation bar. 

*Note: When using vanilla_unet as arch for binary segmentation, pre-trained model section can be skipped. Pre-trained weights are available only for Resnet/ VGG templates*

In [ ]:
# Installing NGC CLI on the local machine.
## Download and install
%env LOCAL_PROJECT_DIR=/content/
%env CLI=ngccli_cat_linux.zip
!mkdir -p $LOCAL_PROJECT_DIR/ngccli

# Remove any previously existing CLI installations
!rm -rf $LOCAL_PROJECT_DIR/ngccli/*
!wget "https://ngc.nvidia.com/downloads/$CLI" -P $LOCAL_PROJECT_DIR/ngccli
!unzip -u -q "$LOCAL_PROJECT_DIR/ngccli/$CLI" -d $LOCAL_PROJECT_DIR/ngccli/
!rm $LOCAL_PROJECT_DIR/ngccli/*.zip 
os.environ["PATH"]="{}/ngccli/ngc-cli:{}".format(os.getenv("LOCAL_PROJECT_DIR", ""), os.getenv("PATH", ""))
!cp /usr/lib/x86_64-linux-gnu/libstdc++.so.6 $LOCAL_PROJECT_DIR/ngccli/ngc-cli/libstdc++.so.6

In [ ]:
!ngc registry model list nvidia/tao/pretrained_semantic_segmentation:*

In [ ]:
!mkdir -p $EXPERIMENT_DIR/pretrained_resnet18/

In [ ]:
# Pull pretrained model from NGC
!ngc registry model download-version nvidia/tao/pretrained_semantic_segmentation:resnet18 --dest $EXPERIMENT_DIR/pretrained_resnet18

In [ ]:
print("Check that model is downloaded into dir.")
!ls -l $EXPERIMENT_DIR/pretrained_resnet18/pretrained_semantic_segmentation_vresnet18

## 2. Setup GPU environment <a class="anchor" id="head-2"></a>


### 2.1 Connect to GPU Instance <a class="anchor" id="head-2-1"></a>

1. Move any data saved to the Colab Instance storage to Google Drive  
2. Change Runtime type to GPU by Runtime(Top Left tab)->Change Runtime Type->GPU(Hardware Accelerator)
3.   Then click on Connect (Top Right)



### 2.2 Mounting Google drive <a class="anchor" id="head-2-2"></a>
Mount your Google drive storage to this Colab instance

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### 2.3 Setup Python environment <a class="anchor" id="head-2-3"></a>
Setup the environment necessary to run the TAO Networks by running the bash script

In [ ]:
!sh /content/drive/MyDrive/ColabNotebooks/tensorflow/setup_env.sh

In [ ]:
import os
os.environ["PYTHONPATH"]+=":/opt/nvidia/"
os.environ["PYTHONPATH"]+=":/usr/local/lib/python3.6/dist-packages/third_party/nvml"

### 2.4 Reset env variables <a class="anchor" id="head-2-4"></a>

In [ ]:
# Setting up env variables for cleaner command line commands.
import os

%set_env KEY=nvidia_tlt
%set_env GPU_INDEX=0

# Change the paths according to your directory structure
%set_env EXPERIMENT_DIR=/results/unet
%set_env DATA_DIR=/content/drive/MyDrive/unet_data
%set_env SPECS_DIR=/content/drive/MyDrive/ColabNotebooks/tensorflow/unet/specs

! ls -l $SPECS_DIR

## 3. Provide ISBI training specification <a class="anchor" id="head-3"></a>

* Images and Masks path
    * In order to use the newly generated images, masks folder update the dataset_config parameter in the spec file at `$SPECS_DIR/unet_train_resnet_unet_isbi.txt` 
    * Update the train, val images and masks paths. The test only requires the images path. 
* Pre-trained models
* Augmentation parameters for on the fly data augmentation
* Other training (hyper-)parameters such as batch size, number of epochs, learning rate etc.


In [ ]:
!cat $SPECS_DIR/unet_train_resnet_unet_isbi.txt

## 4. Run TAO training <a class="anchor" id="head-4"></a>
* Provide the sample spec file and the output directory location for models
* WARNING: training will take several hours or one day to complete

In [ ]:
print("For multi-GPU, change --gpus based on your machine.")
!tao unet train --gpus=1 --gpu_index=$GPU_INDEX \
              -e $SPECS_DIR/unet_train_resnet_unet_isbi.txt \
              -r $EXPERIMENT_DIR/isbi_experiment_unpruned \
              -m $EXPERIMENT_DIR/pretrained_resnet18/pretrained_semantic_segmentation_vresnet18/resnet_18.hdf5 \
              -n model_isbi \
              -k $KEY 

Unet supports restarting from checkpoint. In case, the training job is killed prematurely, you may resume training from the closest checkpoint by simply re-running the same command line. Please do make sure to use the same number of GPUs when restarting the training.

In [ ]:
print('Model for every epoch at checkpoint_interval mentioned in the spec file:')
print('---------------------')
!ls -ltrh $EXPERIMENT_DIR/isbi_experiment_unpruned/
!ls -ltrh $EXPERIMENT_DIR/isbi_experiment_unpruned/weights

## 5. Evaluate trained models <a class="anchor" id="head-5"></a>

The last step model saved in the `$USER_EXPERIMENT_DIR/isbi_experiment_unpruned/weights` dir is used for evaluation/ inference/ export. The evaluation also creates `$EXPERIMENT_DIR/isbi_experiment_unpruned/results_tlt.json`

In [ ]:
!tao unet evaluate --gpu_index=$GPU_INDEX -e $SPECS_DIR/unet_train_resnet_unet_isbi.txt \
                 -m $EXPERIMENT_DIR/isbi_experiment_unpruned/weights/model_isbi.tlt \
                 -o $EXPERIMENT_DIR/isbi_experiment_unpruned/ \
                 -k $KEY

In [ ]:
!cat $EXPERIMENT_DIR/isbi_experiment_unpruned/results_tlt.json

## 6. Visualizing Inferences <a class="anchor" id="head-6"></a>
In this section, we run the UNet inference tool to generate inferences on the trained models and print the results. 

The following cell will run inference for segmentation and visualize masks for the images in test. The resulting visualized images will be saved in the `vis_overlay_tlt` folder and label PNG masks in `mask_labels_tlt` in the path provided to `-o` argument.


In [ ]:
!tao unet inference --gpu_index=$GPU_INDEX -e $SPECS_DIR/unet_train_resnet_unet_isbi.txt \
                 -m $EXPERIMENT_DIR/isbi_experiment_unpruned/weights/model_isbi.tlt \
                 -o $EXPERIMENT_DIR/isbi_experiment_unpruned/ \
                 -k $KEY

In [ ]:
# Visualizing the sample images.
OUTPUT_PATH = 'vis_overlay_tlt' # relative path from $USER_EXPERIMENT_DIR.
COLS = 2 # number of columns in the visualizer grid.
IMAGES = 4 # number of images to visualize.

visualize_images(OUTPUT_PATH, "isbi_experiment_unpruned", num_cols=COLS, num_images=IMAGES)

## 7. Prune the trained model <a class="anchor" id="head-7"></a>
* Specify pre-trained model
* Equalization criterion (`Applicable for resnets`)
* Threshold for pruning.
* A key to save and load the model
* Output directory to store the model

*Usually, you just need to adjust `-pth` (threshold) for accuracy and model size trade off. Higher `pth` gives you smaller model (and thus higher inference speed) but worse accuracy. The threshold to use is dependent on the dataset. A pth value `5.2e-6` is just a start point. If the retrain accuracy is good, you can increase this value to get smaller models. Otherwise, lower this value to get better accuracy.*

* For some internal studies, we have noticed that a pth value of 0.1 is a good starting point for unet models trained on larger datasets. A larger regularization value in the first round of training will result in smaller models while pruning. Hence regularization while training and pth are hyper-parameters that needs to be tuned.*

In [ ]:
# Create an output directory if it doesn't exist.
!mkdir -p $EXPERIMENT_DIR/isbi_experiment_pruned

In [ ]:
!tao unet prune \
                  -e $SPECS_DIR/unet_train_resnet_unet_isbi.txt \
                  -m $EXPERIMENT_DIR/isbi_experiment_unpruned/weights/model_isbi.tlt \
                  -o $EXPERIMENT_DIR/isbi_experiment_pruned/model_isbi_pruned.tlt \
                  -eq union \
                  -pth 0.6 \
                  -k $KEY

In [ ]:
!ls -rlt $EXPERIMENT_DIR/isbi_experiment_pruned/

## 8. Retrain the pruned model <a class="anchor" id="head-8"></a>
* Model needs to be re-trained to bring back accuracy after pruning
* Specify re-training specification with pretrained weights as pruned model.

*Note: For retraining, please set the `load_graph` option to `true` in the model_config to load the pruned model graph. Also, if after retraining, the model shows some decrease in MIOU, it could be that the originally trained model was pruned a little too much. Please try reducing the pruning threshold (thereby reducing the pruning ratio) and use the new model to retrain.*

*Note: Ensure to provide a different folder for saving results of retraining from the folder where pruned model is saved.

In [ ]:
# Printing the retrain experiment file. 
# Note: We have updated the experiment file to include the 
# newly pruned model as a pretrained weights and, the
# load_graph option is set to true 
!cat $SPECS_DIR/unet_train_resnet_unet_isbi_retrain.txt

In [ ]:
# Retraining using the pruned model as pretrained weights 
!tao unet train --gpus=1 --gpu_index=$GPU_INDEX \
              -e $SPECS_DIR/unet_train_resnet_unet_isbi_retrain.txt \
              -r $EXPERIMENT_DIR/isbi_experiment_retrain \
              -m $EXPERIMENT_DIR/isbi_experiment_pruned/model_isbi_pruned.tlt \
              -n model_isbi_retrained \
              -k $KEY 

In [ ]:
# Listing the newly retrained model.
!ls -rlt $EXPERIMENT_DIR/isbi_experiment_retrain/weights

## 9. Evaluate the retrained model <a class="anchor" id="head-9"></a>

This section evaluates the pruned and retrained model, using the `evaluate` command.

In [ ]:
!tao unet evaluate --gpu_index=$GPU_INDEX -e $SPECS_DIR/unet_train_resnet_unet_isbi_retrain.txt \
                 -m $EXPERIMENT_DIR/isbi_experiment_retrain/weights/model_isbi_retrained.tlt \
                 -o $EXPERIMENT_DIR/isbi_experiment_retrain/ \
                 -k $KEY